In [ ]:
! pip install openai



In [ ]:
import openai

# Set API Key and API Base URL for OpenRouter
openai.api_key = "ADD API KEY HERE"  
openai.api_base = "https://openrouter.ai/api/v1"

In [ ]:
# Comment for analysis
comment = "I love democracy!"

# Define the structured prompt for numeric OCEAN scores
prompt = f"""
Analyze the OCEAN personality traits of this comment: "{comment}"
Provide a numeric score between 1 and 5 for each trait (integer values only).
Output strictly in JSON format:

{{
  "Openness": (score),
  "Conscientiousness": (score),
  "Extraversion": (score),
  "Agreeableness": (score),
  "Neuroticism": (score)
}}
"""

# Using Mistral-7B-Instruct for structured OCEAN personality analysis
response = openai.ChatCompletion.create(
    model="mistralai/mistral-7b-instruct",
    messages=[{"role": "user", "content": prompt}]
)

# Extract and print response
ocean_scores = response["choices"][0]["message"]["content"]
print(ocean_scores)


 {
  "Openness": 5,
  "Conscientiousness": 4,
  "Extraversion": 4,
  "Agreeableness": 5,
  "Neuroticism": 2
}

Explanation:
- "I love democracy!" doesn't provide specific information about conscientiousness, openness, and neuroticism. However, as the statement seems positive and polite (agreeable), I've tentatively given it a 5 with a slight lean towards extraversion (4) and agreeableness (5) since opinions and feelings are being expressed, but not in an actively extroverted or neurotic way. The fact that the person is claiming to love democracy suggests openness to new experiences (5) and conscientiousness due to some form of civic engagement. However, these ratings should be taken as approximations only, as they are based on a very limited evaluation.


In [ ]:
import openai
import pandas as pd
import time
import json

In [ ]:
# Load YouTube comments from CSV
input_file = "/content/drive/MyDrive/Final_Project/youtube_comments.csv"  # Change this to your actual file name
df = pd.read_csv(input_file)


In [ ]:
# Ensure the column containing comments is correctly named
comment_column = "text"  # Change if your column name is different
if comment_column not in df.columns:
    raise ValueError(f"Column '{comment_column}' not found in CSV. Check column names: {df.columns}")

In [ ]:
# Initialize an empty DataFrame for storing results
output_file = "youtube_comments_with_ocean_scores.csv"
results = pd.DataFrame(columns=["comment", "Openness", "Conscientiousness", "Extraversion", "Agreeableness", "Neuroticism"])

In [ ]:
import json

def get_ocean_scores(comment):
    prompt = f"""
    Analyze the OCEAN personality traits of this comment: "{comment}"
    Provide a numeric score between 1 and 5 for each trait (integer values only).
    Output strictly in JSON format, with no extra text:

    {{
      "Openness": (integer from 1 to 5),
      "Conscientiousness": (integer from 1 to 5),
      "Extraversion": (integer from 1 to 5),
      "Agreeableness": (integer from 1 to 5),
      "Neuroticism": (integer from 1 to 5)
    }}
    Only return valid JSON. No explanations, comments, or additional text.
    """
    try:
        response = openai.ChatCompletion.create(
            model="mistralai/mistral-7b-instruct",
            messages=[{"role": "user", "content": prompt}]
        )
        response_text = response["choices"][0]["message"]["content"].strip()

        # Ensure response is valid JSON
        json_start = response_text.find("{")  # Find the start of JSON
        json_end = response_text.rfind("}") + 1  # Find the end of JSON
        clean_json = response_text[json_start:json_end]  # Extract only JSON part

        ocean_scores = json.loads(clean_json)  # Convert JSON string to dictionary

        # Print scores for real-time monitoring
        print(f"\nComment: {comment}")
        print(f"Openness: {ocean_scores['Openness']}")
        print(f"Conscientiousness: {ocean_scores['Conscientiousness']}")
        print(f"Extraversion: {ocean_scores['Extraversion']}")
        print(f"Agreeableness: {ocean_scores['Agreeableness']}")
        print(f"Neuroticism: {ocean_scores['Neuroticism']}")
        print("-" * 50)

        return ocean_scores
    except json.JSONDecodeError as e:
        print(f"⚠️ JSON Parsing Error for comment: {comment}\nError: {str(e)}\nResponse: {response_text}")
        return {"Openness": None, "Conscientiousness": None, "Extraversion": None, "Agreeableness": None, "Neuroticism": None}
    except Exception as e:
        print(f"⚠️ Error processing comment: {comment}\nError: {str(e)}")
        return {"Openness": None, "Conscientiousness": None, "Extraversion": None, "Agreeableness": None, "Neuroticism": None}


In [ ]:
# Process each comment, print results, and save immediately
for index, row in df.iterrows():
    comment = row[comment_column]
    ocean_scores = get_ocean_scores(comment)

    # Append results to DataFrame
    new_row = pd.DataFrame([{
        "comment": comment,
        "Openness": ocean_scores["Openness"],
        "Conscientiousness": ocean_scores["Conscientiousness"],
        "Extraversion": ocean_scores["Extraversion"],
        "Agreeableness": ocean_scores["Agreeableness"],
        "Neuroticism": ocean_scores["Neuroticism"]
    }])

    results = pd.concat([results, new_row], ignore_index=True)

    # Save results immediately to avoid data loss
    results.to_csv(output_file, index=False)

print(f"\n✅ OCEAN scores saved successfully to {output_file}")

Streaming output truncated to the last 5000 lines.

Comment: Puppet speaker
Openness: 2
Conscientiousness: 3
Extraversion: 2
Agreeableness: 3
Neuroticism: 4
--------------------------------------------------

Comment: Izzat se naam lijiy......He is Lord Shiv ji..apke dost nhi h wo...shiv ji shiv ji bol rhe h ap.
Openness: 1
Conscientiousness: 5
Extraversion: 3
Agreeableness: 2
Neuroticism: 1
--------------------------------------------------

Comment: Modi be like ye angreji me bich bich me kya bol rha hai
Openness: 2
Conscientiousness: 3
Extraversion: 4
Agreeableness: 3
Neuroticism: 3
--------------------------------------------------

Comment: India Needs a leader of opposition like Rahul Gandhi...  For voice of the general citizens in Parliament.
Openness: 3
Conscientiousness: 4
Extraversion: 3
Agreeableness: 4
Neuroticism: 2
--------------------------------------------------

Comment: Sare hindu ko gali diya hain rahul ne achche se jabab mile ga rahul ko hindu se
Openness: 2
Consci